# Tweet and spaCy EDA
This notebook looks at the capabilities of spaCy in extracting location in formation from our live tweets.

In [3]:
import jsonpickle
import json
import datetime
import time

import regex as re
import pandas as pd
import spacy

In [12]:
live_tweets = pd.read_csv('../data/tweets/current_tweets.csv')

In [13]:
live_tweets.head()

,id,username,date,text,hashtags,geo,type
0,1194676579634470912,CaltransDist6,2019-11-13 18:01:07+00:00,Notice a bad pothole or litter? Report Califor...,NaN,NaN,official
1,1194417493013516288,CaltransDist6,2019-11-13 00:51:36+00:00,EAST MADERA COUNTY/OAKHURST AREA: Caltrans cre...,NaN,NaN,official
2,1194299055335194624,CaltransDist6,2019-11-12 17:00:58+00:00,Caltrans' Volunteer Program can assist in fulf...,NaN,NaN,official
3,1194169567175532544,CaltransDist6,2019-11-12 08:26:26+00:00,FRESNO: Eastbound 180 NOW OPEN at 41 following...,NaN,NaN,official
4,1194142636254171136,CaltransDist6,2019-11-12 06:39:25+00:00,FRESNO: Eastbound 180 CLOSED at 41 due to acci...,NaN,NaN,official


In [8]:
nlp = spacy.load("en_core_web_sm")

In [16]:
live_tweets[live_tweets['username']=="CaltransDist6"]['text']

0      Notice a bad pothole or litter? Report Califor...
1      EAST MADERA COUNTY/OAKHURST AREA: Caltrans cre...
2      Caltrans' Volunteer Program can assist in fulf...
3      FRESNO: Eastbound 180 NOW OPEN at 41 following...
4      FRESNO: Eastbound 180 CLOSED at 41 due to acci...
                             ...                        
195    CITY OF FRESNO: Tonight, Monday, June 24 from ...
196    CITY OF FRESNO: Southbound State Route 99 nort...
197    KERN COUNTY: The big rig has been cleared from...
198    KERN COUNTY: Traffic is currently backed up on...
199    TULARE COUNTY:State Route 137 between Rd 44 an...
Name: text, Length: 200, dtype: object

# Training spaCy
Training spaCy is an extensive process. It is possible to add known entities to its basic vocabulary. To do this, we scraped every street name in California, but the syntax for appending those terms to the spaCy library was beyond what our time allowed.

In future iterations, we would either need to build up this process to create a robust library of street names, interstates, state routes, exits and cross-streets for whatever region we are looking at, or we would need to find a workaround that did not rely on spaCy.

In [36]:
# spaCy with its basic english package, is able to recognize locations like 
# "California" and "Asia" it was unable to pull streenames for us.
doc = nlp(live_tweets['text'][1])

spacy.displacy.render(doc, style='ent', jupyter=True)

In [38]:
streets = pd.read_csv('../data/scrapes/california_street_names.csv')

## spaCy training example from website.
Example of training spaCy's named entity recognizer, starting off with an
existing model or a blank model.

For more details, see the documentation:
* Training: https://spacy.io/usage/training
* NER: https://spacy.io/usage/linguistic-features#named-entities

Compatible with: spaCy v2.0.0+
Last tested with: v2.1.0
"""

```python
from __future__ import unicode_literals, print_function

import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding

TRAIN_DATA = listo

@plac.annotations(
    model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
    output_dir=("Optional output directory", "option", "o", Path),
    n_iter=("Number of training iterations", "option", "n", int),
)
def main(model=None, output_dir=None, n_iter=100):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")

    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model
        if model is None:
            nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)

    # test the trained model
    for text, _ in TRAIN_DATA:
        doc = nlp(text)
        print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
        print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        for text, _ in TRAIN_DATA:
            doc = nlp2(text)
            print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
            print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])
```

In [108]:
listo = []
for index, item in TRAIN_DATA.iterrows():
    listo.append((item['TEXT'], {"entities": [(item['START'], item['END'], "STREET")]}))
listo

[('1st Street', {'entities': [(0, 10, 'STREET')]}),
 ('2nd Street', {'entities': [(0, 10, 'STREET')]}),
 ('3rd Street', {'entities': [(0, 10, 'STREET')]}),
 ('4th Street', {'entities': [(0, 10, 'STREET')]}),
 ('5th Street', {'entities': [(0, 10, 'STREET')]}),
 ('6th Street', {'entities': [(0, 10, 'STREET')]}),
 ('8th Street', {'entities': [(0, 10, 'STREET')]}),
 ('9th Street', {'entities': [(0, 10, 'STREET')]}),
 ('Adams Street', {'entities': [(0, 12, 'STREET')]}),
 ('Adelphian Way', {'entities': [(0, 13, 'STREET')]}),
 ('Admiralty Lane', {'entities': [(0, 14, 'STREET')]}),
 ('Alameda Avenue', {'entities': [(0, 14, 'STREET')]}),
 ('Alameda Road', {'entities': [(0, 12, 'STREET')]}),
 ('Alaska Packer Place', {'entities': [(0, 19, 'STREET')]}),
 ('Alta Vis', {'entities': [(0, 8, 'STREET')]}),
 ('Amber Isle', {'entities': [(0, 10, 'STREET')]}),
 ('Anchor Way', {'entities': [(0, 10, 'STREET')]}),
 ('Anderson Road', {'entities': [(0, 13, 'STREET')]}),
 ('Annapolis Circle', {'entities': [(0, 

In [96]:
TRAIN_DATA = streets[['TEXT', 'ENTITY', 'START','END', 'LABEL']]

## Summary
spaCy has a very nice display feature where you can see the color-coded entities. Again, it is pulling large districts and counties, but is not finding street names. We would like to continue exploring this method, but may need to move on to new methods.